In [2]:
from load_results import load_result_dataset

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'

final_data = load_result_dataset(pn1, pn2)

../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [11]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index

# df['total_acc'] = df['clean_acc'] + df['Linf_acc']  # Sum of scores per dataset

import pandas as pd

df = pd.DataFrame(final_data)

nan_percentage = (df.isna().sum().sum() / df.size) * 100
print(f"Percentage of NaN values: {nan_percentage:.2f}%")

df = df[ df.model_size == 2 ]

grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'],
    columns='dataset',
    # dropna=False
)

# Rename column levels
grouped_df.columns.set_names(["metric", "dataset"], inplace=True)

# Swap column levels → dataset becomes level 0, metric becomes level 1
grouped_df.columns = grouped_df.columns.swaplevel(0, 1)

# (Optional) Sort columns so that all metrics are grouped within each dataset
grouped_df = grouped_df.sort_index(axis=1, level=0)

grouped_df.to_csv("./{}.csv".format(pn1))

grouped_df

Percentage of NaN values: 23.77%


dataset                                                       caltech101  \
metric                                                            L1_acc   
backbone                                        loss_function              
CLIP-convnext_base_w-laion2B-s13B-b82K          CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
CLIP-convnext_base_w-laion_aesthetic-s13B-b82K  CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
coatnet_2_rw_224.sw_in12k                       CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
coatnet_2_rw_224.sw_in12k_ft_in1k               CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
convnext_base.clip_laion2b_augreg_ft_in12k_in1k CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
convnext_base.fb_in1k                           CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
convnext_base.fb_in22k                          CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
convnext_base.fb_in22k_ft_in1k                  CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
eva02_base_patch14_224.mim_in22k                CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
robust_convnext_base                            CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
robust_vit_base_patch16_224                     CLASSIC_AT      0.760000   
                                                TRADES_v2       0.755000   
swin_base_patch4_window7_224.ms_in22k_ft_in1k   CLASSIC_AT           NaN   
                                                TRADES_v2            NaN   
vit_base_patch16_224.augreg_in1k                CLASSIC_AT      0.681000   
                                                TRADES_v2       0.717333   
vit_base_patch16_224.augreg_in21k               CLASSIC_AT      0.658667   
                                                TRADES_v2       0.733000   
vit_base_patch16_224.augreg_in21k_ft_in1k       CLASSIC_AT      0.730000   
                                                TRADES_v2       0.666000   
vit_base_patch16_224.mae                        CLASSIC_AT      0.587000   
                                                TRADES_v2       0.155000   
vit_base_patch16_clip_224.laion2b               CLASSIC_AT      0.609000   
                                                TRADES_v2       0.757333   
vit_base_patch16_clip_224.laion2b_ft_in1k       CLASSIC_AT      0.460000   
                                                TRADES_v2       0.776000   

dataset                                                                  \
metric                                                           L2_acc   
backbone                                        loss_function             
CLIP-convnext_base_w-laion2B-s13B-b82K          CLASSIC_AT          NaN   
                                                TRADES_v2           NaN   
CLIP-convnext_base_w-laion_aesthetic-s13B-b82K  CLASSIC_AT          NaN   
                                                TRADES_v2           NaN   
coatnet_2_rw_224.sw_in12k                       CLASSIC_AT          NaN   
                                                TRADES_v2           NaN   
coatnet_2_rw_224.sw_in12k_ft_in1k               CLASSIC_AT          NaN   
                                                TRADES_v2           NaN   
convnext_base.clip_laion2b_augreg_ft_in12k_in1k CLASSIC_AT          NaN   
                                                TRADES_v2           NaN   
convnext_base.fb_in1k                           CLASSIC_AT  

In [20]:

# --- existing shortcuts ----------------------------------------------------
sum_scores  = grouped_df.xs('sum',  level=1, axis=1)     # (rows × 6 datasets)
geom_scores = grouped_df.xs('geom', level=1, axis=1)     # (rows × 6 datasets)

#--- 1. add the TOTAL aggregates you already computed ----------------------
grouped_df[('TOTAL', 'score_sum')]  = sum_scores.sum(axis=1)
grouped_df[('TOTAL', 'score_geom')] = geom_scores.sum(axis=1)

# --- 2. count NaNs across datasets -----------------------------------------
grouped_df[('TOTAL', 'nan_sum_cnt')]  = sum_scores.isna().sum(axis=1)
grouped_df[('TOTAL', 'nan_geom_cnt')] = geom_scores.isna().sum(axis=1)

# --- 3. rank as before ------------------------------------------------------
df_sorted = grouped_df.sort_values(('TOTAL', 'score_sum'), ascending=False)

df_sorted

KeyError: 'sum'